# Visão Assistentes

**Objetivo:** consolidar chamadas (`tabe_calls`), pesquisa URA (`tabe_pesq_ura`) e dimensão de assistentes (`dim_assistentes`) **D‑1** em uma visão diária por assistente, materializada em `prd.g_dm_callcenter.visao_assistentes` com `replaceWhere` por `DT_REFE`.

## Imports


In [0]:
from pyspark.sql import functions as F, Window as W
from datetime import datetime, timedelta

## Parâmetros


In [0]:
# Data de referência (D‑1 por padrão)
odate = (datetime.now() - timedelta(days=1)).date()

# Tabelas origens
nm_tabe_calls = 'prd.s_dm_callcenter.tabe_calls'
nm_tabe_pesq_ura  = 'prd.s_dm_callcenter.tabe_pesq_ura'
nm_dim_assist  = 'prd.b_dm_callcenter.dim_assistentes'

# Tabela final
nm_tabe_final = 'prd.g_dm_callcenter.visao_assistentes'

## 1) Chamadas
- **IN_RECH:** cliente que volta a ligar em ≤24h (cálculo por janela)


In [0]:
df_tabe_calls = (
    # Base origem
    spark.table(nm_tabe_calls)

    # Filtrando o dia do dado
    .filter(F.col('DT_INIC') == odate)

    # Criando coluna de chamada atendida
    .withColumn('IN_CHAM_ATEN', F.lit(1))

    # Criando a coluna de rechamada, sempre que o cliente liga novamente em menos de 24 horas
    .withColumn('IN_RECH', 
        F.when(
            (F.col('ID_CHAM') != (F.lead('ID_CHAM',1).over(W.partitionBy('ID_CLIE').orderBy('DH_INIC'))))
            & (F.unix_timestamp(F.col('DH_INIC')) - (F.lead(F.unix_timestamp(F.col('DH_INIC')),1).over(W.partitionBy('ID_CLIE').orderBy('DH_INIC'))) <= 86400) 
        ,F.lit(1)).otherwise(F.lit(0))
    )
)

## 2) Pesquisa URA
- **Sinalização NPS:** `IN_PRMT` (promotor), `IN_NTRO` (neutro) e `IN_DETR` (detrator).
> *Regra adotada*: promotor `>= 6`, neutro `== 5`, detrator `<= 4`


In [0]:
df_tabe_pesq_ura = (
    # Base origem
    spark.table(nm_tabe_pesq_ura)

    # Filtrando o dia do dado
    .filter(F.col('DT_ENVI') == odate)

    # Criando as colunas de PROMOTOR, NEUTRO e DETRATOR
    .withColumn('IN_PRMT', F.when(F.col('VL_NOTA') >= 6,F.lit(1)).otherwise(F.lit(0)))
    .withColumn('IN_NTRO', F.when(F.col('VL_NOTA') == 5,F.lit(1)).otherwise(F.lit(0)))
    .withColumn('IN_DETR', F.when(F.col('VL_NOTA') <= 4,F.lit(1)).otherwise(F.lit(0)))
)


## 3) Dimensão de Assistentes

In [0]:
rename_cols_assist = {
    'identificadorAssistente': 'ID_ASST',
    'area': 'DS_AREA',
    'nomeAssistente': 'NM_ASST',
    'nomeSupervisor': 'NM_SVSP',
    'nomeCoordenador': 'NM_CORD',
    'nomeGerente': 'NM_GERN',
    'nomeSuperintendente': 'NM_SUPT'
}

df_dim_assist = (
    # Base origem
    spark.table(nm_dim_assist)

    # Renomeandos os campos para uso posterior
    .withColumnsRenamed(rename_cols_assist)
)

## 4) Join + Métricas agregadas

In [0]:
result = df_dim_assist.alias('assist') \
    .join(df_tabe_calls.alias('calls'), on='ID_ASST', how='inner') \
    .join(df_tabe_pesq_ura.alias('pesq'), on='ID_CHAM', how='left') \
    .withColumn('TEMPO_ATENDIMENTO', F.unix_timestamp(F.col('DH_FIM')) - F.unix_timestamp(F.col('DH_INIC'))) \
    .select(
        'calls.CD_PERI','DT_INIC', 'DS_AREA', 'ID_ASST', 'NM_ASST', 'NM_SVSP', 'NM_CORD', 'NM_GERN', 'NM_SUPT',
        'IN_CHAM_ATEN','IN_TRAF','IN_TRAF_INDV','IN_RECH','TEMPO_ATENDIMENTO', 'IN_PRMT', 'IN_NTRO', 'IN_DETR'
    ) \
    .withColumnRenamed('DT_INIC', 'DT_REFE') \
    .groupby('CD_PERI', 'DT_REFE', 'DS_AREA', 'ID_ASST', 'NM_ASST', 'NM_SVSP', 'NM_CORD', 'NM_GERN', 'NM_SUPT') \
    .agg(
        F.sum('IN_CHAM_ATEN').alias('QT_CHAM_ATEN'),
        F.sum('IN_TRAF').alias('QT_TRAF'),
        F.round(F.sum('IN_TRAF') / F.sum('IN_CHAM_ATEN'),2).alias('PC_TRAF'),
        F.sum('IN_TRAF_INDV').alias('QT_TRAF_INDV'),
        F.round(F.sum('IN_TRAF_INDV') / F.sum('IN_CHAM_ATEN'),2).alias('PC_TRAF_INDV'),
        F.sum('IN_RECH').alias('QT_RECH'),
        F.round(F.sum('IN_RECH') / F.sum('IN_CHAM_ATEN'),2).alias('PC_RECH'),
        F.round(F.sum('TEMPO_ATENDIMENTO') / F.sum('IN_CHAM_ATEN'),2).alias('VL_TEMP_MEDI_OPER'),
        F.sum('IN_PRMT').alias('QT_PRMT'),
        F.sum('IN_NTRO').alias('QT_NTRO'),
        F.sum('IN_DETR').alias('QT_DETR'),
        F.round(
            (F.sum('IN_PRMT') - F.sum('IN_DETR')) / 
            (F.sum('IN_PRMT') + F.sum('IN_NTRO') + F.sum('IN_DETR')) * 100
        ,2).alias('VL_NPS')
    )

## 5) Criação da tabela destino
Cria a tabela e aplica `CLUSTER BY (CD_PERI, DT_REFE, ID_ASST)`

In [0]:
if not spark.catalog.tableExists(nm_tabe_final):
    empty = spark.createDataFrame([], result.schema)
    (empty.write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(nm_tabe_final))
    spark.sql(f"ALTER TABLE {nm_tabe_final} CLUSTER BY (CD_PERI, DT_REFE, ID_ASST)")

result.write \
    .mode('overwrite') \
    .option('replaceWhere', f"DT_REFE = '{odate}'") \
    .saveAsTable(nm_tabe_final)